In [1]:
from __future__ import print_function, division
from builtins import range
import numpy as np

In [2]:
GAMMA = 0.9
ALPHA = 0.1
POSSIBLE_ACTIONS = ('U','D','L','R') 

In [3]:
#Environment
class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
        
    def set(self, rewards, actions):
        self.rewards = rewards
        self.actions = actions
        
    def set_state(self, state):
        self.i = state[0]
        self.j = state[1]
        
    def current_state(self):
        return(self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions 
    
    def move(self, action):
        if action in self.actions[(self.i, self.j)]:
            if action == 'U':
                self.i -= 1
            elif action =='D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -= 1 
        
        return self.rewards.get((self.i, self.j), 0)
        
    
    def undo_move(self):
        if action == 'U':
            self.i += 1
        elif action =='D':
            self.i -= 1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1 
        
        assert(self.current_state() in self.all_states())
                
            
    def game_over(self):
        return (self.i, self.j) not in self.actions
    
    
    def all_states(self):
        return set().union(self.actions.keys(), self.rewards.keys())

    

In [4]:
def standard_grid():
    # .  .  .  1
    # .  x  .  -1
    # s  .  .  .
    
    # 1, -1 are rewards for the corresponding positions
    # 'x'  not allowed
    # 's'  start state
    grid = Grid(3, 4, (2,0))
    rewards = {(0,3) : 1, (1,3) : -1}
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
    }
    grid.set(rewards, actions)
    return grid

In [5]:
def print_values(V, grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") 
        print("")

In [6]:
def print_policy(P, grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            a = P.get((i,j), ' ')
            print("  %s  |" % a, end="")
        print("")

In [7]:
def random_action(a, eps=0.1):
    p = np.random.random()
    if p < eps:
        return np.random.choice(POSSIBLE_ACTIONS)
    else:
        return a

In [8]:
def play_grid_world(grid, policy):
    s = (2,0)
    grid.set_state(s)
    states_and_rewards = [(s,0)]
    
    while not grid.game_over():
        a = policy[s]
        a = random_action(a)
        r = grid.move(a)
        states_and_rewards.append((s,r))
        s = grid.current_state()
    states_and_rewards.append((s,r))
    return states_and_rewards

In [10]:
if __name__ == '__main__':
    grid = standard_grid()
    
    print("Rewards")
    print_values(grid.rewards, grid)
    
    
    # policy
    policy = {
        (0,0):'R',
        (1,0):'U',
        (2,0):'U',
        (0,1):'R',
        (1,2):'R',
        (2,1):'R',
        (0,2):'R',
        (2,2):'R',
        (2,3):'U',
    }
    
    V = {}
    states = grid.all_states()
    for s in states:
        V[s] = 0
            
            
    for e in range(5000):
        # play an episode
        states_and_rewards = play_grid_world(grid, policy)
        
        for t in range(len(states_and_rewards) - 1):
            s1, r1 = states_and_rewards[t]
            s2, r2 = states_and_rewards[t+1]
            V[s1] = V[s1] + ALPHA * (r1 + GAMMA* V[s2] - V[s1])
        
    print("\n\nvalues")
    print_values(V, grid)
    
    print("\n\npolicy")
    print_policy(policy, grid)

Rewards
---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|


values
---------------------------
 0.76| 0.87| 0.99| 0.00|
---------------------------
 0.66| 0.00|-1.00| 0.00|
---------------------------
 0.53|-0.80|-0.89|-0.99|


policy
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  U  |  R  |  R  |  U  |
